# Step 1

In this step we will first load the information using panda



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
df = pd.read_csv('Heart_Disease_Prediction.csv')
df.head()


In [9]:
df["Heart Disease"] = df["Heart Disease"].map({"presence": 1, "Absence": 0})
df.head()


# EDA


In [5]:
df.describe()

# Prep 70/30


In [ ]:
features = ['Age', 'Chest pain type', 'BP', 'Max HR', 'ST depression', 'Cholesterol', 'Number of vessels fluro']
X = df[features]
Y = df['Heart Disease']

### Target Distribution

In [ ]:
import matplotlib.pyplot as plt

# Plot distribution of the target variable
plt.figure(figsize=(6, 4))
df['Heart Disease'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Heart Disease Class Distribution')
plt.xlabel('Class (0=Absence, 1=Presence)')
plt.ylabel('Count')
plt.show()

### Splitting and Normalization (70/30)

In [ ]:
# Randomize and Split (70/30)
np.random.seed(42)
indices = np.random.permutation(len(X))
test_size = int(len(X) * 0.3)
test_indices = indices[:test_size]
train_indices = indices[test_size:]

X_train = X.iloc[train_indices]
y_train = Y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = Y.iloc[test_indices]

# Normalize (Z-score) using parameters from training set
mean = X_train.mean()
std = X_train.std()

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)
X_train.head()

# Step 2: Implement Basic Logistic Regression
Sigmoid, cost (binary cross-entropy), GD (gradients, updates; track costs).

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w, b):
    m = len(y)
    z = np.dot(X, w) + b
    f_wb = sigmoid(z)
    # Clip to avoid log(0)
    epsilon = 1e-15
    f_wb = np.clip(f_wb, epsilon, 1 - epsilon)
    cost = -1/m * np.sum(y * np.log(f_wb) + (1 - y) * np.log(1 - f_wb))
    return cost

def compute_gradient(X, y, w, b):
    m = len(y)
    z = np.dot(X, w) + b
    f_wb = sigmoid(z)
    err = f_wb - y
    dj_dw = 1/m * np.dot(X.T, err)
    dj_db = 1/m * np.sum(err)
    return dj_dw, dj_db

def gradient_descent(X, y, w_in, b_in, alpha, num_iters):
    w = w_in
    b = b_in
    J_history = []
    
    for i in range(num_iters):
        dj_dw, dj_db = compute_gradient(X, y, w, b)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        if i % 100 == 0 or i == num_iters-1:
            cost = compute_cost(X, y, w, b)
            J_history.append(cost)
            # print(f"Iteration {i}: Cost {cost}")
            
    return w, b, J_history

In [ ]:
# Train on full train set
np.random.seed(1)
initial_w = np.zeros(X_train.shape[1])
initial_b = 0
iterations = 1000
alpha = 0.01

w_final, b_final, J_hist = gradient_descent(X_train.to_numpy(), y_train.to_numpy(), initial_w, initial_b, alpha, iterations)

# Plot cost vs iterations
plt.plot(range(0, iterations + 1, 100), J_hist)
plt.title('Cost vs Iterations')
plt.ylabel('Cost')
plt.xlabel('Iterations (per 100)')
plt.show()

print(f"Final Cost: {J_hist[-1]}")
print(f"Final Weights: {w_final}")
print(f"Final Bias: {b_final}")

In [ ]:
# Predict and Evaluate
def predict(X, w, b):
    z = np.dot(X, w) + b
    return (sigmoid(z) >= 0.5).astype(int)

y_pred_train = predict(X_train.to_numpy(), w_final, b_final)
y_pred_test = predict(X_test.to_numpy(), w_final, b_final)

def calculate_metrics(y_true, y_pred, set_name):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    accuracy = (tp + tn) / len(y_true)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"--- {set_name} Metrics ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print()

calculate_metrics(y_train, y_pred_train, "Train")
calculate_metrics(y_test, y_pred_test, "Test")

# Step 3: Visualize Decision Boundaries
Select ≥3 feature pairs, train 2D models, and plot.

In [ ]:
def plot_decision_boundary(w, b, X, y, feature_names):
    # Plot data points
    plt.figure(figsize=(8, 6))
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Absence (0)', marker='o')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Presence (1)', marker='x')
    
    # Calculate boundary
    # w1*x1 + w2*x2 + b = 0 => x2 = -(w1*x1 + b) / w2
    x1_min, x1_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    x1_vals = np.linspace(x1_min, x1_max, 100)
    
    if w[1] != 0:
        x2_vals = -(w[0] * x1_vals + b) / w[1]
        plt.plot(x1_vals, x2_vals, 'k--', label='Decision Boundary')
    else:
        plt.axvline(x=-b/w[0], color='k', linestyle='--', label='Decision Boundary')
        
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])
    plt.legend()
    plt.title(f'Decision Boundary: {feature_names[0]} vs {feature_names[1]}')
    plt.show()

feature_pairs = [
    ['Age', 'Cholesterol'],
    ['BP', 'Max HR'],
    ['ST depression', 'Number of vessels fluro']
]

for pair in feature_pairs:
    # Subset data
    X_pair_train = X_train[pair].to_numpy()
    
    # Train model on just these 2 features
    w_pair_init = np.zeros(2)
    w_pair, b_pair, _ = gradient_descent(X_pair_train, y_train.to_numpy(), w_pair_init, 0, 0.01, 1000)
    
    # Plot
    plot_decision_boundary(w_pair, b_pair, X_pair_train, y_train.to_numpy(), pair)

# Step 4: Repeat with Regularization (L2)

In [ ]:
def compute_cost_reg(X, y, w, b, lambda_reg):
    m = len(y)
    cost_no_reg = compute_cost(X, y, w, b)
    reg_term = (lambda_reg / (2 * m)) * np.sum(w**2)
    return cost_no_reg + reg_term

def compute_gradient_reg(X, y, w, b, lambda_reg):
    m = len(y)
    dj_dw, dj_db = compute_gradient(X, y, w, b)
    dj_dw += (lambda_reg / m) * w
    return dj_dw, dj_db

def gradient_descent_reg(X, y, w, b, alpha, num_iters, lambda_reg):
    J_history = []
    for i in range(num_iters):
        dj_dw, dj_db = compute_gradient_reg(X, y, w, b, lambda_reg)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        if i % 100 == 0:
            J_history.append(compute_cost_reg(X, y, w, b, lambda_reg))
            
    return w, b, J_history

lambdas = [0, 0.001, 0.01, 0.1, 1]

print("--- Regularization Tuning ---")
for lam in lambdas:
    w_reg, b_reg, _ = gradient_descent_reg(X_train.to_numpy(), y_train.to_numpy(), 
                                           np.zeros(X_train.shape[1]), 0, 
                                           0.01, 1000, lam)
    
    y_pred_tun = predict(X_test.to_numpy(), w_reg, b_reg)
    acc = np.mean(y_pred_tun == y_test)
    print(f"Lambda: {lam}, Test Accuracy: {acc:.4f}")